# Unfolding Symmetric Modes

This example demonstrates the `unfold` function, which reconstructs the full mode profile from solutions computed with symmetry boundary conditions.

We use a silicon channel waveguide with oxide cladding. By exploiting the symmetry of the fundamental TE mode, we compute the mode on only the right half of the waveguide using a PMC boundary condition on the west edge. The `unfold` function then reconstructs the complete mode profile.

In [ ]:
# Enable automatic reloading of modules (IPython only)
%reload_ext autoreload
%autoreload 2 

# load required modules
import modesolver as mode
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

In [ ]:
# Refractive indices:
n1 = 1.46           # SiO2 lower cladding
n2 = 3.50           # Silicon core
n3 = 1.46           # Upper cladding

# Vertical dimensions:
h1 = 500            # lower cladding (nm), will be stretched
h2 = 200            # silicon core (nm)
h3 = 500            # upper cladding (nm), will be stretched

# Horizontal dimensions:
w = 400             # full width (nm)
side = 500          # Space on side, will be stretched

# Grid size:
dx = 2.5            # grid size (x)
dy = dx             # grid size (y)

# Build waveguide mesh for RIGHT HALF only (west edge at x=0)
x, y, xc, yc, nx, ny, eps, edges = mode.waveguidemesh(
    [n1, n2, n3], [h1, h2, h3], h2, w/2, side, dx, dy, return_edges=True
)

# Stretch cladding layers (NORTH, SOUTH, EAST) by 1.5X
x, y, xc, yc, dx, dy = mode.stretchmesh(x, y, [200, 200, 200, 0], [1.5, 1.5, 1.5, 1])

In [ ]:
wavelength = 1550   # vacuum wavelength (nm)
nmodes = 1          # number of modes to compute

# Compute quasi-TE mode on right half with PMC on west boundary
bc = '000M'  # North=0, South=0, East=0, West=M (PMC)
neff, hx, hy, hzj = mode.wgmodes(
    wavelength, n2, nmodes, dx, dy, eps=eps, boundary=bc
)
print(f"neff (TE mode, half domain) = {neff:.6f}")

In [ ]:
# Unfold fields across west boundary
x_full, y_full, xc_full, yc_full, hx_full, hy_full, hzj_full = mode.unfold(
    x, y, xc, yc, hx, hy, hzj, boundary=bc, unfold='W'
)

# Also unfold eps for plotting waveguide structure
x_eps, y_eps, xc_eps, yc_eps, eps_full = mode.unfold(
    x, y, xc, yc, boundary=bc, unfold='W', eps=eps
)

print(f"Original domain: x ∈ [{x.min():.3f}, {x.max():.3f}]")
print(f"Unfolded domain: x ∈ [{x_full.min():.3f}, {x_full.max():.3f}]")

In [ ]:
# Plot TE mode fields
levels = np.arange(-45, 0, 3)
figTE, (axHx, axHy, axHz) = plt.subplots(1, 3, figsize=(12, 5))

axHx.pcolormesh(x_full, y_full, abs(hx_full), cmap='jet', vmax=1)
axHx.contour(x_full, y_full, 20*np.log10(abs(hx_full)), levels, 
             colors='white', negative_linestyles='solid', linewidths=1)
axHx.set_aspect('equal')
axHx.axis([min(x_full), max(x_full), min(y_full), max(y_full)])
axHx.set_xlabel('x')
axHx.set_ylabel('y')
axHx.set_title(f'Hx (TE Mode)\n$ n_{{eff}} = {neff:.6f} $')
lHx = LineCollection(edges, colors='black', linewidths=1)
axHx.add_collection(lHx)

axHy.pcolormesh(x_full, y_full, abs(hy_full), cmap='jet', vmax=1)
axHy.contour(x_full, y_full, 20*np.log10(abs(hy_full)), levels, 
             colors='white', negative_linestyles='solid', linewidths=1)
axHy.set_aspect('equal')
axHy.axis([min(x_full), max(x_full), min(y_full), max(y_full)])
axHy.set_xlabel('x')
axHy.set_ylabel('y')
axHy.set_title(f'Hy (TE Mode)\n$ n_{{eff}} = {neff:.6f} $')
lHy = LineCollection(edges, colors='black', linewidths=1)
axHy.add_collection(lHy)

axHz.pcolormesh(x_full, y_full, abs(hzj_full), cmap='jet', vmax=1)
axHz.contour(x_full, y_full, 20*np.log10(abs(hzj_full)), levels, 
             colors='white', negative_linestyles='solid', linewidths=1)
axHz.set_aspect('equal')
axHz.axis([min(x_full), max(x_full), min(y_full), max(y_full)])
axHz.set_xlabel('x')
axHz.set_ylabel('y')
axHz.set_title(f'Hz*j (TE Mode)\n$ n_{{eff}} = {neff:.6f} $')
lHz = LineCollection(edges, colors='black', linewidths=1)
axHz.add_collection(lHz)

plt.show()